In [1]:
###############################
#                             #
#        ENCODE FACTORS       #
#                             #
###############################
import pandas as pd
# performs label encoding
from sklearn.preprocessing import LabelEncoder
def label_encoding(df_train, df_valid, df_test):
    
    factors = df_train.select_dtypes('object').columns
    
    lbl = LabelEncoder()

    for f in factors:        
        lbl.fit(list(df_train[f].values) + list(df_valid[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_valid[f] = lbl.transform(list(df_valid[f].values))
        df_test[f]  = lbl.transform(list(df_test[f].values))

    return df_train, df_valid, df_test

import numpy as np
def reduce_mem_usage(df, verbose = True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
train_ = pd.read_csv('../raw/training.csv')

train_ = train_[-train_['hospital_death'].isnull()]
y = train_['hospital_death']

In [7]:
############ LIBRARIES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import random
import multiprocessing
import pickle

import scipy.stats
from scipy.stats import rankdata

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors

In [8]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 23):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 23
seed_everything(seed)

In [13]:
### IMPORT OOF PREDS

# which model to stack?
#model = 'lgb'

# threshold
#min_auc = 90

# list names
names = sorted(os.listdir('../oof_preds'))
names.remove('.ipynb_checkpoints',)
#names = [n for n in names if int(n[n.rindex('_')+1:-7]) > min_auc]
#names = [s for s in names if model in s]

# preprocessing loop
for name in names:

    # load preds
    tmp_tr = pd.read_csv('../oof_preds/'   + str(name))
    tmp_te = pd.read_csv('../submissions/' + str(name))

    # sort preds by ID
    #tmp_tr = tmp_tr.sort_values('encounter_id')
    #tmp_te = tmp_te.sort_values('encounter_id')

    
    # cbind data
    if name == names[0]:  
        
        tmp_tr.columns = ['encounter_id', name]    
        tmp_te.columns = ['encounter_id', name]    
        train = tmp_tr 
        test  = tmp_te
        
    else:
        
        del tmp_tr['encounter_id'], tmp_te['encounter_id']
        tmp_tr.columns = [name]    
        tmp_te.columns = [name]    
        train = pd.concat([train, tmp_tr], axis = 1)
        test  = pd.concat([test,  tmp_te], axis = 1)
    
    '''
    del tmp_tr['TransactionID'], tmp_te['TransactionID']
    tmp_tr.columns = [name]    
    tmp_te.columns = [name]    
    train = pd.concat([train, tmp_tr], axis = 1)
    test  = pd.concat([test,  tmp_te], axis = 1)
    '''
        
# display information
print('- Train shape:', train.shape)
print('- Test shape:',  test.shape)

FileNotFoundError: [Errno 2] File ../submissions/lgb_v90_seed111_90669.csv does not exist: '../submissions/lgb_v90_seed111_90669.csv'

In [ ]:
# drop bad features
excluded_feats = ['encounter_id']
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

In [ ]:
### PARAMETERS

# settings
cores = -1
seed  = 999

# cross-validation
num_folds = 10
shuffle   = True

# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)

In [ ]:
# placeholders
clfs = []
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])
importances  = pd.DataFrame()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
### CROSS-VALIDATION LOOP
cv_start  = time.time()
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    
    # label encoding
    trn_x, val_x, test_x = label_encoding(trn_x, val_x, test[features])
    
    # print data dimensions
    print('Data shape:', trn_x.shape, val_x.shape)
           
    # train Ridge
    clf = LogisticRegression(random_state=seed).fit(trn_x, trn_y)

    # predictions
    preds_oof[val_idx]    = clf.predict_proba(val_x)[:, 1]
    preds_test           += clf.predict_proba(test_x)[:, 1] / folds.n_splits 

    
    # print performance
    print('----------------------')
    print('FOLD%2d: AUC = %.6f' % (n_fold + 1, roc_auc_score(y[val_idx], preds_oof[val_idx])))
    print('----------------------')
    print('')
        
    # clear memory
    #del trn_x, trn_y, val_x, val_y
    #gc.collect()
    
    
# print overall performance    
cv_perf = roc_auc_score(y, preds_oof)
print('--------------------------------')
print('- OOF AUC = %.6f' % cv_perf)
print('- CV TIME = {:.2f} min'.format((time.time() - cv_start) / 60))
print('--------------------------------')

In [15]:
############ RECHECK PERFORMANCE  

# check performance
print(np.round(roc_auc_score(y, preds_oof), 5))

0.97293


In [16]:
# file name
model = 'stack_ridge_'
perf  = str(round(cv_perf, 6))[2:7]
name  = model + str(len(features)) + '_' + perf
name

'stack_ridge_31_97292'

In [17]:
# export OOF preds
oof = pd.DataFrame({'encounter_id': train['encounter_id'], 'hospital_death': preds_oof})
oof.to_csv('./oof_preds/' + str(name) + '.csv', index = False)
oof.head()

,encounter_id,hospital_death
0,66154,0.020426
1,114252,0.091363
2,119783,0.027105
3,79267,0.022203
4,92056,0.028702


In [21]:

# export submission
sub = pd.DataFrame({'encounter_id': test['encounter_id'], 'hospital_death': preds_test})
sub.to_csv('./submissions/' + str(name) + '.csv', index = False)
sub.head()

,encounter_id,hospital_death
0,2,0.023944
1,5,0.028942
2,7,0.026765
3,8,0.073585
4,10,0.215931


In [ ]:
sub.head(39308).to_csv('./submissions/' + str(name) + '.csv', index = False)